In [66]:
import re
import numpy as np
from itertools import combinations
import sys

In [67]:
sys.setrecursionlimit(200)

In [231]:
with open('../input/day24.txt', 'r') as f:
    lines = f.read()

lines[:10]

'x00: 1\nx01'

In [232]:
inits_r, connects_r = lines.split('\n\n')

In [259]:
def init_wires(inits_r):
    inits = {}
    for init in inits_r.split('\n'):
        wire, bit = init.split(': ')
        inits[wire] = int(bit)

    return inits



In [260]:
def init_connects(connects_r):
    connects = {}
    for connect in connects_r.split('\n'):
        parsed_connect = re.findall('\w+', connect)
        target = parsed_connect[-1]
        inputs = parsed_connect[0], parsed_connect[2]
        logic = parsed_connect[1]
        connects[target] = (inputs, logic)
    return connects



In [268]:
def eval_wire(wire, inits, connects):
    if wire in inits.keys():
        return inits[wire]
        
    inputs = connects[wire]
    logic = inputs[1]
    wires = [wire for wire in inputs[0]]
    if logic == 'AND':
        return eval_wire(wires[0], inits, connects) & eval_wire(wires[1], inits, connects)

    elif logic == 'OR':
        return eval_wire(wires[0], inits, connects) | eval_wire(wires[1], inits, connects)

    elif logic == 'XOR':
        return eval_wire(wires[0], inits, connects) ^ eval_wire(wires[1], inits, connects)


In [269]:
z_wires = np.sort([z for z in connects.keys() if z[0] == 'z'])
z_wires

array(['z00', 'z01', 'z02', 'z03', 'z04', 'z05', 'z06', 'z07', 'z08',
       'z09', 'z10', 'z11', 'z12', 'z13', 'z14', 'z15', 'z16', 'z17',
       'z18', 'z19', 'z20', 'z21', 'z22', 'z23', 'z24', 'z25', 'z26',
       'z27', 'z28', 'z29', 'z30', 'z31', 'z32', 'z33', 'z34', 'z35',
       'z36', 'z37', 'z38', 'z39', 'z40', 'z41', 'z42', 'z43', 'z44',
       'z45'], dtype='<U3')

In [270]:
output = []
inits = init_wires(inits_r).copy()
connects = init_connects(connects_r).copy()
for wire in z_wires:
    output.insert(0, str(eval_wire(wire, inits, connects)))
int("".join(output), 2)

59336987801432

## Part 2 with some help from reddit community; learned about adders and used some hints and sample code

In [298]:
def validate_output(wire_num, connects):

    inits = init_wires(inits_r).copy()
    for key in inits.keys():
        inits[key] == 0
        
    for x in (0, 1):
        for y in (0, 1):
            for carry in (0, 1):
                init_x = [x] + [0] * (wire_num - 1)
                init_y = [y] + [0] * (wire_num - 1)
                
                if wire_num > 0: 
                    init_x = [0] * (wire_num - 1) + [carry] + init_x 
                    init_y = [0] * (wire_num - 1) + [carry] + init_y 
                    
                if wire_num == 0 and carry == 1:
                    continue 

                for i, (bx, by) in enumerate(zip(init_x, init_y)):
                    inits["x" + str(i).zfill(2)] = bx
                    inits["y" + str(i).zfill(2)] = by
                
                z = eval_wire("z" + str(wire_num).zfill(2), inits, connects)
                if z != (x + y + carry) % 2:
                    return False
    
    return True

In [299]:
def get_first_failure(connects, lowest_failure):
    for i in range(lowest_failure, 45):
        if not validate_output(i, connects):
            return i

    return -1

In [300]:
def swap_connects(w1, w2, connects):
    c1 = connects[w1]
    c2 = connects[w2]

    connects[w1] = c2
    connects[w2] = c1

In [301]:
def find_swap_candidates(connects, lowest_failure):
    swap_candidates = []
    new_lowest = lowest_failure
    while new_lowest != -1 or len(swap_candidates) < 4:
        # Loop until we've tried every pair - or there are no more failures
        for w1 in connects: 
            for w2 in connects: 
                if w1 == w2: 
                    continue
                
                sorted_pair = tuple(sorted((w1, w2)))
                if sorted_pair in swap_candidates:
                    continue
                
                swap_connects(w1, w2, connects)
                try:
                    new_lowest = get_first_failure(connects, lowest_failure)
                    if new_lowest == -1 or new_lowest > lowest_failure:
                        lowest_failure = new_lowest
                        swap_candidates.append(sorted_pair)
                        break
                except RecursionError as e:
                    pass 
                
                swap_connects(w1, w2, connects)
            else:
                continue
            
            break
        
        if len(swap_candidates) == 0:
            print("dafuq")
            break
            
    return swap_candidates, connects

In [302]:
connects = init_connects(connects_r)
    
lowest_failure = get_first_failure(connects, 0)
swaps, local_connects = find_swap_candidates(connects, lowest_failure)

In [303]:
",".join(sorted([item for pair in swaps for item in pair]))

'ctg,dmh,dvq,rpb,rpv,z11,z31,z38'